In [ ]:
# dataset
# https://www.kaggle.com/datasets/mathurinache/math-dataset
# !kaggle datasets download -d mathurinache/math-dataset

In [1]:
!kaggle datasets download -d mathurinache/math-dataset
!unzip math-dataset.zip
!pip install datasets
!pip install transformers
!pip install peft
!pip install trl

from IPython import display
display.clear_output()

In [57]:
from datasets import load_dataset
import transformers
from IPython.display import display, Latex
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from trl import SFTTrainer

In [3]:
LIMIT = 1000
CONTEXTWINDOW = 512

In [5]:
!huggingface-cli login

n
n

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
EOS_TOKEN = tokenizer.eos_token
tokenizer.pad_token = EOS_TOKEN
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
def tokenize_function(examples):

   return tokenizer(examples['text'],
                    padding="max_length",
                    max_length=CONTEXTWINDOW,
                    truncation=True,
                    return_tensors='pt')

In [6]:
alpaca_prompt = """Below is a Maths Question. Write an answer that appropriately completes the request.

### Question:
{}

### Answer:
"""

def generate_prompt(examples):
    problems  = examples["problem"]
    solutions = examples["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        texts.append(
            alpaca_prompt.format(problem) + solution + EOS_TOKEN
        )
    return { "text" : texts, }

In [29]:
dataset = load_dataset("json", data_dir='./MATH')
dataset = dataset.filter(lambda example, index: index < LIMIT, with_indices=True)
dataset = dataset.map(generate_prompt, batched=True)
dataset = dataset.map(tokenize_function, batched=True).with_format("torch")

Resolving data files:   0%|          | 0/7500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5000 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [61]:
print("Question:\n")
display(Latex(dataset['train'][0]['problem']))
print("Answer:\n")
display(Latex(dataset['train'][0]['solution']))

Question:



<IPython.core.display.Latex object>

Answer:



<IPython.core.display.Latex object>

In [49]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [16]:
lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [17]:
# Loading in 8 bit ..."
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [30]:
output_dir = "cp"
per_device_train_batch_size = 2
gradient_accumulation_steps = 8
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 5e-4
max_grad_norm = 0.3
max_steps = 50
warmup_ratio = 0.03
evaluation_strategy="steps"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            fp16=True,
            gradient_checkpointing=True,
            eval_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

In [31]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field='text',
    peft_config=lora_config,
    max_seq_length=1024,
    args=training_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda'

In [32]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [14]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.1 MB/s eta 0:00:00


In [33]:
trainer.train()
trainer.save_model(f"{output_dir}/final")

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
10,0.980900,1.022671
20,0.971300,1.014077
30,0.960100,1.008036
40,0.974600,1.002159
50,0.959000,0.998023


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [49]:
""" Sample """

text = alpaca_prompt.format(dataset["test"][0]['problem'])
input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()

generation_output = model.generate(
      input_ids=input_ids,
      max_new_tokens=100,
      temperature=0.7,
      eos_token_id=tokenizer.eos_token_id,
    )

gen_ans = tokenizer.decode(generation_output[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [59]:
display(Latex(dataset["test"][0]['solution']))

<IPython.core.display.Latex object>

In [60]:
display(Latex(gen_ans.split("### Answer:\n")[1]))

<IPython.core.display.Latex object>

In [54]:
model.push_to_hub("MESSItom/math-Llama-3.2-1B") # Online saving
tokenizer.push_to_hub("MESSItom/math-Llama-3.2-1B") # Online saving

adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MESSItom/math-Llama-3.2-1B/commit/398e6c65dfefc2e065b9226f826d2d478cbe1cbc', commit_message='Upload tokenizer', commit_description='', oid='398e6c65dfefc2e065b9226f826d2d478cbe1cbc', pr_url=None, pr_revision=None, pr_num=None)